In [ ]:
from google.colab import drive



drive.mount('/content/drive')

MessageError: Error: credential propagation was unsuccessful

In [ ]:
pip install kaggle

In [ ]:
! mkdir -p ~/.kaggle/

! mv /content/kaggle.json ~/.kaggle/

! chmod 600 /root/.kaggle/kaggle.json

! kaggle datasets download -d sunilthite/ovarian-cancer-classification-dataset

100% 3.31G/3.31G [02:56<00:00, 21.5MB/s]
100% 3.31G/3.31G [02:56<00:00, 20.1MB/s]


In [ ]:
import zipfile
def unzip_folder(zip_file_path, extract_folder):
    with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
        zip_ref.extractall(extract_folder)


zip_file_path = '/content/extracted-images.zip'
extract_folder = '/content/Datasets_Ovarian/ImgData'

unzip_folder(zip_file_path, extract_folder)


In [ ]:
import os

def delete_excess_images(folder_path, max_images=155):
    try:
        image_files = [file for file in os.listdir(folder_path) if file.lower().endswith(('.png', '.jpg', '.jpeg'))]
        num_images = len(image_files)

        if num_images > max_images:
            excess_files = image_files[max_images:]
            for file in excess_files:
                file_path = os.path.join(folder_path, file)
                os.remove(file_path)
            print(f"{num_images - max_images} excess images deleted in folder: {folder_path}")
        else:
            print(f"No excess images found in folder: {folder_path}")

        print(f"Total images in folder: {num_images}")

    except Exception as e:
        print(f"Error occurred: {str(e)}")

folder_path = '/content/Datasets_Ovarian/ImgData/train/EC'
delete_excess_images(folder_path, max_images=2369)


3881 excess images deleted in folder: /content/Datasets_Ovarian/ImgData/train/EC
Total images in folder: 6250


In [ ]:
import os

def count_images_in_folders(root_folder):
    try:
        for dirpath, dirnames, filenames in os.walk(root_folder):
            for dirname in dirnames:
                folder_path = os.path.join(dirpath, dirname)
                image_files = [file for file in os.listdir(folder_path) if file.lower().endswith(('.png', '.jpg', '.jpeg'))]
                num_images = len(image_files)
                print(f"Folder: {dirname}, Total images: {num_images}")

    except Exception as e:
        print(f"Error occurred: {str(e)}")

root_folder = '/content/Datasets_Ovarian/ImgData/train'
count_images_in_folders(root_folder)


Folder: MC, Total images: 2491
Folder: EC, Total images: 2369
Folder: CC, Total images: 2345
Folder: LGSC, Total images: 2720
Folder: HGSC, Total images: 2222


In [ ]:
# Training InceptionResnetV2

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import PIL
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential

batch_size = 4
img_height = 180
img_width = 180
data_dir='/content/Datasets_Ovarian/ImgData/train'

In [ ]:

train_ds = tf.keras.utils.image_dataset_from_directory(
  data_dir,
  validation_split=0.2,
  subset="training",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)
val_ds = tf.keras.utils.image_dataset_from_directory(
  data_dir,
  validation_split=0.2,
  subset="validation",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

Found 12147 files belonging to 5 classes.
Using 9718 files for training.
Found 12147 files belonging to 5 classes.
Using 2429 files for validation.


In [ ]:
class_names = train_ds.class_names
print(class_names)

['CC', 'EC', 'HGSC', 'LGSC', 'MC']


In [ ]:
AUTOTUNE = tf.data.AUTOTUNE

train_ds = train_ds.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)
normalization_layer = layers.Rescaling(1./255)
normalized_ds = train_ds.map(lambda x, y: (normalization_layer(x), y))
image_batch, labels_batch = next(iter(normalized_ds))
first_image = image_batch[0]
print(np.min(first_image), np.max(first_image))

0.12766863 0.98692775


In [ ]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.applications import EfficientNetB4
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.metrics import Accuracy


num_classes = len(class_names)
base_model = EfficientNetB4(weights=None, include_top=False, input_shape=(img_height, img_width, 3))
base_model.trainable = True
model = Sequential([
    base_model,
    GlobalAveragePooling2D(),
    Dense(128, activation='relu'),
    Dense(num_classes, activation='softmax')
])

model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

epochs=50
history = model.fit(
  train_ds,
  validation_data=val_ds,
  epochs=epochs
)


Epoch 1/50


/usr/local/lib/python3.10/dist-packages/keras/src/backend.py:5727: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Softmax activation and thus does not represent logits. Was this intended?
  output, from_logits = _get_logits(


2430/2430 [==============================] - 278s 86ms/step - loss: 1.5593 - accuracy: 0.2810 - val_loss: 6.9671 - val_accuracy: 0.2268
Epoch 2/50
2430/2430 [==============================] - 209s 86ms/step - loss: 1.4896 - accuracy: 0.3127 - val_loss: 1.4450 - val_accuracy: 0.3014
Epoch 3/50
2430/2430 [==============================] - 209s 86ms/step - loss: 1.4555 - accuracy: 0.3338 - val_loss: 1.8418 - val_accuracy: 0.1721
Epoch 4/50
2430/2430 [==============================] - 207s 85ms/step - loss: 1.3685 - accuracy: 0.3900 - val_loss: 1.7461 - val_accuracy: 0.3306
Epoch 5/50
2430/2430 [==============================] - 210s 86ms/step - loss: 1.2636 - accuracy: 0.4459 - val_loss: 1.9794 - val_accuracy: 0.3491
Epoch 6/50
2430/2430 [==============================] - 207s 85ms/step - loss: 1.2030 - accuracy: 0.4687 - val_loss: 1.8201 - val_accuracy: 0.2956
Epoch 7/50
2430/2430 [==============================] - 210s 86ms/step - loss: 1.1655 - accuracy: 0.4966 - val_loss: 1.2680 - val

KeyboardInterrupt: 

In [ ]:
model.save('/content/drive/MyDrive/Datasets_Ovarian/Img_Data/IP_OV/ovarian_EfficientNetB4_.h5')


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, roc_curve, confusion_matrix, matthews_corrcoef


acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_range = range(epochs)

plt.figure(figsize=(4, 3))
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy in Ovarian Using EfficientNetB2 Model')

plt.savefig('/content/drive/MyDrive/Datasets_Ovarian/IMG-Data/EFF_OV/Training_Val_Acc_ovarian_cancer.png')

plt.figure(figsize=(4, 3))
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss in Ovarian Using InceptionResnetV2 Model')
plt.savefig('/content/drive/MyDrive/Datasets_Ovarian/IMG-Data/EFF_OV/Training_Val_loss_ovarian_cancer.png')
plt.show()


predictions = model.predict(val_ds)

true_labels = []
for images, labels in val_ds:
    true_labels.extend(labels.numpy())

true_labels = np.array(true_labels)
predicted_labels = np.argmax(predictions, axis=1)

accuracy = accuracy_score(true_labels, predicted_labels)
precision = precision_score(true_labels, predicted_labels, average='macro')
recall = recall_score(true_labels, predicted_labels, average='macro')
f1 = f1_score(true_labels, predicted_labels, average='macro')
mcc = matthews_corrcoef(true_labels, predicted_labels)
conf_matrix = confusion_matrix(true_labels, predicted_labels)

plt.figure(figsize=(4, 3))
sns.heatmap(conf_matrix, annot=True, fmt="d", cmap="Blues", cbar=False)
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.title('Confusion Matrix for Ovarian using EfficientNetB3')
plt.savefig('/content/drive/MyDrive/Datasets_Ovarian/IMG-Data/EFF_OV/Ovarian_EfficientNetB2_confusion_matrix.png')
plt.show()

auc_roc_classes = roc_auc_score(true_labels, predictions, multi_class='ovr')

plt.figure(figsize=(4, 3))
for i in range(num_classes):
    fpr, tpr, _ = roc_curve(true_labels[:, i], predictions[:, i])
    plt.plot(fpr, tpr, label=f'Class {i} (AUC = {auc_roc_classes[i]:.2f})')
plt.plot([0, 1], [0, 1], linestyle='--', color='gray')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic (ROC) Curve for Ovarian cancer on EfficientNetB2')
plt.legend(loc='lower right')
plt.grid()
plt.savefig('/content/drive/MyDrive/Datasets_Ovarian/Image_Data/EFF_OV/Ovarian_EfficientNetB2_roc_curve.png')
plt.show()

# Print evaluation metrics
print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)
print("MCC:", mcc)

import csv

file_path = '/content/drive/MyDrive/Datasets_Ovarian/IMG-Data/EFF_OV/EFF_evaluation_metrics.csv'




In [ ]:
evaluation_metrics = {
    'Accuracy': accuracy,
    'Precision': precision,
    'Recall': recall,
    'F1 Score': f1,
    'MCC': mcc
}

with open(file_path, mode='w', newline='') as file:
    writer = csv.DictWriter(file, fieldnames=evaluation_metrics.keys())
    writer.writeheader()
    writer.writerow(evaluation_metrics)